# 1. Articles - Data Collection and Cleaning & 2. Merging into Opininon Leaders Datasets

# 1. Data Collection & Cleaning - Articles

In [1]:
import pandas as pd
from datetime import datetime
import requests.auth as auth
import requests
from bs4 import BeautifulSoup
import time
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from collections import Counter
import nltk
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

## 1.1 Articles - 600e

### 1.1.1 Scraping

In [2]:
# open the browser and get the page for the Fiat 600e search
driver = webdriver.Chrome() # Open a new Chrome window
url600 = 'https://news.google.com/search?q=fiat%20600e%20-abarth&hl=en-GB&gl=GB&ceid=GB%3Aen' # URL for Google Trends
driver.get(url600) # Open the URL
soup = BeautifulSoup(driver.page_source, "html.parser") # Parse the HTML of the page

# Find accept cookies button and click it
cookie = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button')
cookie.click()

In [3]:
# Scroll and load more articles
def scroll_and_load(driver, pause_time=2, max_scrolls=20):
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

# Scroll and load all articles
scroll_and_load(driver)

# Parse the loaded page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

In [4]:
#*Retrieve links from Google News*

# Find all the elements containing the URLs of the articles
articles600_links = driver.find_elements(By.XPATH, '//a[@class="WwrzSb"]')

# Extract the URLs from those elements
articles600_urls = [link.get_attribute("href") for link in articles600_links]

# Extract corresponding times of the articles <time class="hvbAAd" datetime="2024-03-05T08:00:00Z">5 Mar</time>
articles600_times = driver.find_elements(By.XPATH, '//time[@class="hvbAAd"]')
articles600_times = [time.get_attribute("datetime") for time in articles600_times]

# Make a list of url and corresponding time
articles600_info = list(zip(articles600_urls, articles600_times))

# Print the URLs
for element in articles600_info:
    print(element)

('https://news.google.com/articles/CBMiaWh0dHBzOi8vd3d3LmF1dG9leHByZXNzLmNvLnVrL2ZpYXQvNjAwLzM2Mzg0MS9maWF0LTYwMGUtbGEtcHJpbWEtbG9uZy10ZXJtLXRlc3QtdXJiYW4tZXYtaGFzLXBsZW50eS1mbGFpctIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-07-22T14:04:01Z')
('https://news.google.com/articles/CBMiT2h0dHBzOi8vYXV0b2dlYXIucHQvZW4vZ2xvYmFsLXNhbGVzLW9mLWZpYXQtaW5jcmVhc2VkLWJ5LTItMi1pbi10aGUtZmlyc3QtaGFsZi_SAQA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-07-30T10:22:58Z')
('https://news.google.com/articles/CBMiNmh0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1yZXZpZXdzL2ZpYXQvNjAwZS9pbnRlcmlvctIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-03-05T08:00:00Z')
('https://news.google.com/articles/CBMikwFodHRwczovL3d3dy50aGlzaXNtb25leS5jby51ay9tb25leS9lbGVjdHJpY2NhcnMvYXJ0aWNsZS0xMzE4MjMxMy9GaWF0LTYwMGUtcmV2aWV3LU5ldy0zM2stZWxlY3RyaWMtZmFtaWx5LWNhci1jb25zaWRlci1jaGVhcGVyLWh5YnJpZC12ZXJzaW9uLmh0bWzSAQA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-03-12T07:00:00Z')
('https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LndhbGxwYXBlci5jb20vdHJhbnNwb3J

In [5]:
# Have a look at the first 4 articles
articles600_info[:4]

[('https://news.google.com/articles/CBMiaWh0dHBzOi8vd3d3LmF1dG9leHByZXNzLmNvLnVrL2ZpYXQvNjAwLzM2Mzg0MS9maWF0LTYwMGUtbGEtcHJpbWEtbG9uZy10ZXJtLXRlc3QtdXJiYW4tZXYtaGFzLXBsZW50eS1mbGFpctIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-07-22T14:04:01Z'),
 ('https://news.google.com/articles/CBMiT2h0dHBzOi8vYXV0b2dlYXIucHQvZW4vZ2xvYmFsLXNhbGVzLW9mLWZpYXQtaW5jcmVhc2VkLWJ5LTItMi1pbi10aGUtZmlyc3QtaGFsZi_SAQA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-07-30T10:22:58Z'),
 ('https://news.google.com/articles/CBMiNmh0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1yZXZpZXdzL2ZpYXQvNjAwZS9pbnRlcmlvctIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-03-05T08:00:00Z'),
 ('https://news.google.com/articles/CBMikwFodHRwczovL3d3dy50aGlzaXNtb25leS5jby51ay9tb25leS9lbGVjdHJpY2NhcnMvYXJ0aWNsZS0xMzE4MjMxMy9GaWF0LTYwMGUtcmV2aWV3LU5ldy0zM2stZWxlY3RyaWMtZmFtaWx5LWNhci1jb25zaWRlci1jaGVhcGVyLWh5YnJpZC12ZXJzaW9uLmh0bWzSAQA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-03-12T07:00:00Z')]

In [38]:
# Check the number of articles
len(articles600_info)

94

In [6]:
# Scrape the text of the articles
articles_600e = []
# Loop through the URLs
for url, ptime in articles600_info:
    driver.get(url)  # Open the URL
    # Accept cookies if the button exists
    try:
        cookie = driver.find_element(By.XPATH, '//button[contains(@class,"VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 XWZjwc")]')
        cookie.click()
        time.sleep(3)  # Wait for the cookies modal to disappear
    except:
        pass  # Do nothing in the except part
    
    time.sleep(3)  # Allow the page some time to fully load
        
    # Parse the HTML of the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
        
    # Find all the elements that are paragraph HTML elements
    paragraphs = soup.find_all('p')
        
    # Extract the text from each paragraph and append each as a separate data point
    for para in paragraphs:
        para_text = para.get_text()
        articles_600e.append([url, ptime, para_text])

# Close the driver
driver.quit()

# Create a DataFrame and save it to a CSV file
articles_600e_original = pd.DataFrame(articles_600e, columns=['URL', 'Published', 'Text'])

print("Scraping complete. Data saved to articles.csv")
print(articles_600e_original.head())

Scraping complete. Data saved to articles.csv
                                                 URL             Published  \
0  https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
1  https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
2  https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
3  https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
4  https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   

                                                Text  
0                                       4.0 out of 5  
1  Fiat does extremely well when it comes to cute...  
2  Put together your ideal combination of element...  
3  I live in London, so all of this makes for exc...  
4  The 600e is Fiat’s new crossover, adding two m...  


### 1.1.2 Cleaning

In [7]:
# make a copy to clean the data
articles_600e_process = articles_600e_original.copy()

In [8]:
# clean duplicates
articles_600e_process = articles_600e_process.drop_duplicates()
articles_600e_process = articles_600e_process.dropna()
# clean rows that have only one word in 'Text' column
articles_600e_process = articles_600e_process[articles_600e_process['Text'].str.split().str.len() > 1]
articles_600e_process.head

<bound method NDFrame.head of                                                     URL             Published  \
0     https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
1     https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
2     https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
3     https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
4     https://news.google.com/articles/CBMiaWh0dHBzO...  2024-07-22T14:04:01Z   
...                                                 ...                   ...   
7797  https://news.google.com/articles/CBMiU2h0dHBzO...  2023-07-05T07:00:00Z   
7798  https://news.google.com/articles/CBMiU2h0dHBzO...  2023-07-05T07:00:00Z   
7800  https://news.google.com/articles/CBMiU2h0dHBzO...  2023-07-05T07:00:00Z   
7801  https://news.google.com/articles/CBMiU2h0dHBzO...  2023-07-05T07:00:00Z   
7802  https://news.google.com/articles/CBMiU2h0dHBzO...  2023-07-05T07:00:00Z  

In [9]:
# clean rows that mention date
articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.contains(r'\d{4}-\d{2}-\d{2}', regex=True)]
# mention date in format Month 01, 2023
articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{2},\s\d{4}', regex=True)]
# mention date in format 01 Month 2023
articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.contains(r'\d{2}\s[a-zA-Z]+\s\d{4}', regex=True)]
articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{1},\s\d{4}', regex=True)]

# Define a list of patterns to filter out
patterns600e = [
    '©', 'Find out more', 'terms of use', 'cookie', 'cookies', 'cookie policy', 'privacy policy', 'policy','privacy', 
    'verify', 'email', 'With your acceptance', 'your changes', 'subscription', 'dialog', 'more by', 
    'all rights', 'all right', 'sorry', 'editor', '.com', 'chat', 'Subscriptions', 'data', 'ISSN', 'Tel', 'subscribe','Subscribing',
    'other forms', 'information', 'Information', 'service can be based', 'company number', 'months ago', 
    'monthly', 'best new', 'worst new', 'Product and service reviews', 'Best websites', 'Best sites','your consumer rights', 
    'insurance and warranty offers','Sell your car today','Browse cars for sale','New car leasing offers',"Check a car's history",
    'Award-winning, independent and impartial advice','Expert Ratings','ExpertImmediate Network', "Which car seat will suit you?", "Website of the Year", "Buying and selling a car",  
    "WhatsApp", "subscriber","Login", "log in","free account", "do not necessarily reflect the views of", "Copy link", "Daily Mail", "mortgage", "Let us help you understand", "Truck","trucking",
    "Yahoo", "Ireland", "Horoscope", "county", "attack", "MISSING PERSON","Offaly","BIG PICTURE","Devolved governments",
    "Irish", "education guidance proposal","Japan", "Shepherd","King George","Paralympic","Taylor Swift",
    "Shania Twain","Leslie Grantham","Emma Thompson","doctors","ambulance","disease","Drugs","Scientists","swimming","hair","skin","Audi","Bugatti","Citroen","per article",
    "spoirt", "Fest","IHF’s","Courtesy", "top stories",   "Wales","election","Roscrea", "elections", "hospital", "victims", "Tractor","championships",
    "Local National School","house","killed",  "Artura Spider?",  "Volkswagen’s new Tiguan",
    "UK drive: Hyundai",  "sun blast", "tractors","Photos by","Orthodontic","Job Alert","permission","browser settings","vendors are registered","Japanese",
    "system supported by","reached your account maximum","Corvette","sign in","MembersClick","JavaScript","content free of charge",
    "Market & Tech Reports","Global automobile production and sales statistics","longest day of the year", "Browse for", "Powered by","Uses other forms of storage",
    "buyers reject lofty prices at time of huge capital outlays"
]


# Apply the filters
for pattern in patterns600e:
    articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.contains(pattern, case=False, na=False)]

# Additional specific filters
articles_600e_process = articles_600e_process[~articles_600e_process['Text'].str.startswith('Car', na=False)]


# clean rows IF (consists of 3 words ) AND (includes  'by')
for index, row in articles_600e_process.iterrows():
    # if there are 3 words in the text and ('by' or 'By') is in the text, drop the row
    if len(row['Text'].split()) == 3 and 'by' in row['Text'].lower():
        articles_600e_process.drop(index, inplace=True)

# drop if only 2 words in the text
articles_600e_process = articles_600e_process[articles_600e_process['Text'].str.split().str.len() > 2]

# clean rows IF (consists of less than 4 words) AND (includes  ':')
for index, row in articles_600e_process.iterrows():
    # if there are less than 4 words in the text and ':' is in the text, drop the row
    if len(row['Text'].split()) <= 6 and ':' in row['Text']:
        articles_600e_process.drop(index, inplace=True)


# clean if mention ;best; more than once in lowred case
for index, row in articles_600e_process.iterrows():
    if row['Text'].lower().count('best') > 1:
        articles_600e_process.drop(index, inplace=True)


# clean if contains 'news; that is not in context 'bad news' or 'good news'
for index, row in articles_600e_process.iterrows():
    if 'news' in row['Text'].lower() and 'bad news' not in row['Text'].lower() and 'good news' not in row['Text'].lower():
        articles_600e_process.drop(index, inplace=True)
# drop if only 2 words in the text
articles_600e_process = articles_600e_process[articles_600e_process['Text'].str.split().str.len() > 3]

In [10]:
print(len(articles_600e_original), 'rows in original dataset')
print(len(articles_600e_original) - len(articles_600e_process), 'rows dropped after cleaning')
print(len(articles_600e_process), 'rows left after cleaning')

7803 rows in original dataset
7066 rows dropped after cleaning
737 rows left after cleaning


In [49]:
# Combine paragraphs of the same article
# Group by 'URL' and concatenate text in the same group and keep all columns
articles600_concatenated = articles_600e_process.groupby('URL').agg({'Text': ' '.join, 'Published': 'first'}).reset_index()

# Display the first few rows to verify
print('Number of rows in 600e -Articles: ', len(articles600_concatenated))
print('Fisrt 5 rows in 600e - Articles: \n')
print(articles600_concatenated.head())

# close the browser
driver.quit()

Number of rows in 600e -Articles:  88
Fisrt 5 rows in 600e - Articles: 

                                                 URL  \
0  https://news.google.com/articles/CBMiHmh0dHBzO...   
1  https://news.google.com/articles/CBMiJ2h0dHBzO...   
2  https://news.google.com/articles/CBMiJ2h0dHBzO...   
3  https://news.google.com/articles/CBMiKWh0dHBzO...   
4  https://news.google.com/articles/CBMiNGh0dHBzO...   

                                                Text             Published  
0  JAKARTA - Fiat recently revealed their latest ...  2023-12-29T11:00:24Z  
1  Customers asked for more crossovers, and Fiat ...  2023-08-21T07:00:00Z  
2  \nRelatively affordable to buy. Very useable 2...  2023-09-27T15:21:33Z  
3  4.0 out of 5 The ‘Frappuccino' of the small el...  2023-09-22T09:41:02Z  
4  There are two specifications available on the ...  2024-03-05T08:00:00Z  


In [12]:
# Rename the columns for consistency
# rename URL to ID
articles600_concatenated.rename(columns={'URL': 'ID'}, inplace=True)
# Rename Published to Date
articles600_concatenated.rename(columns={'Published': 'Date'}, inplace=True)
# Date to YYYY-MM-DD without time dt.strftime('%Y-%m-%d')
articles600_concatenated['Date'] = pd.to_datetime(articles600_concatenated['Date']).dt.strftime('%Y-%m-%d')

In [13]:
# saving the cleaned data
articles600e = articles600_concatenated.copy()

## 1.2 Articles - 500e

## 1.2.1 Scraping

In [14]:
# Open a new Chrome window for Fiat 500e search
driver = webdriver.Chrome() # Open a new Chrome window
url500 = "https://news.google.com/search?q=fiat%20500e%20-abarth&hl=en-GB&gl=GB&ceid=GB%3Aen" # URL for Google Trends
driver.get(url500) # Open the URL
soup = BeautifulSoup(driver.page_source, "html.parser") # Parse the HTML of the page

# Find accept cookies button and click it
cookie = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button')
cookie.click()

In [15]:
# Scroll and load all articles
scroll_and_load(driver)

# Parse the loaded page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

In [16]:
#*Retrieve links from Google News*

# Find all the elements containing the URLs of the articles
articles500_links = driver.find_elements(By.XPATH, "//a[@class='WwrzSb']")

# Extract the URLs from those elements
articles500_urls = [link.get_attribute('href') for link in articles500_links]

# Extract corresponding times of the articles <time class="hvbAAd" datetime="2024-03-05T08:00:00Z">5 Mar</time>
articles500_times = driver.find_elements(By.XPATH, "//time[@class='hvbAAd']")
articles500_times = [time.get_attribute('datetime') for time in articles500_times]

# Make a list of url and corresponding time
articles500_info = list(zip(articles500_urls, articles500_times))

# Print the URLs
for element in articles500_info:
    print(element)

('https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNpbmNoLmNvLnVrL2d1aWRlcy9jaG9vc2luZy1hLWNhci93aHktYXJlLWZpYXQtNTAwZS1zby1jaGVhcNIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-07-29T10:06:58Z')
('https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmF1dG9leHByZXNzLmNvLnVrL2ZpYXQvNTAwLzM2Mzg1My9saW1pdGVkLWVkaXRpb24tZmlhdC01MDBlLWdpb3JnaW8tYXJtYW5pLWVkaXRpb24tb25lLWZhc2hpb25pc3Rhc9IBAA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-07-23T11:32:45Z')
('https://news.google.com/articles/CBMilQFodHRwczovL2NhcmRlYWxlcm1hZ2F6aW5lLmNvLnVrL3B1Ymxpc2gvanVsLTI3LXBheS1oaWtlcy1mb3ItcHVibGljLXNlY3Rvci1leHBlY3RlZC1pbnRlcmVzdC1yYXRlLWRlY2lzaW9uLW5leHQtd2Vlay1hcm1hbmktZmlhdC01MDBlLXJldmVhbGVkLzMwNTY2NdIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2024-07-27T05:59:31Z')
('https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5zaHJvcHNoaXJlc3Rhci5jb20vbmV3cy9tb3RvcnMvMjAyNC8wNy8yNi9maWF0LWNlbGVicmF0ZXMtaXRzLTEyNXRoLWFubml2ZXJzYXJ5LXdpdGgtZXhjbHVzaXZlLWdpb3JnaW8tYXJtYW5pLTUwMGUtbW9kZWwv0gEA?hl=en-GB&gl=GB&ceid=GB%3Aen', '2

In [17]:
# Have a look at the first 4 articles
articles500_info[:4]

[('https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNpbmNoLmNvLnVrL2d1aWRlcy9jaG9vc2luZy1hLWNhci93aHktYXJlLWZpYXQtNTAwZS1zby1jaGVhcNIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-07-29T10:06:58Z'),
 ('https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmF1dG9leHByZXNzLmNvLnVrL2ZpYXQvNTAwLzM2Mzg1My9saW1pdGVkLWVkaXRpb24tZmlhdC01MDBlLWdpb3JnaW8tYXJtYW5pLWVkaXRpb24tb25lLWZhc2hpb25pc3Rhc9IBAA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-07-23T11:32:45Z'),
 ('https://news.google.com/articles/CBMilQFodHRwczovL2NhcmRlYWxlcm1hZ2F6aW5lLmNvLnVrL3B1Ymxpc2gvanVsLTI3LXBheS1oaWtlcy1mb3ItcHVibGljLXNlY3Rvci1leHBlY3RlZC1pbnRlcmVzdC1yYXRlLWRlY2lzaW9uLW5leHQtd2Vlay1hcm1hbmktZmlhdC01MDBlLXJldmVhbGVkLzMwNTY2NdIBAA?hl=en-GB&gl=GB&ceid=GB%3Aen',
  '2024-07-27T05:59:31Z'),
 ('https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5zaHJvcHNoaXJlc3Rhci5jb20vbmV3cy9tb3RvcnMvMjAyNC8wNy8yNi9maWF0LWNlbGVicmF0ZXMtaXRzLTEyNXRoLWFubml2ZXJzYXJ5LXdpdGgtZXhjbHVzaXZlLWdpb3JnaW8tYXJtYW5pLTUwMGUtbW9kZWwv0gEA?hl=en-GB&gl=GB&ceid

In [39]:
# Check the number of articles
len(articles500_info)

99

In [18]:
articles_500e = []
# Loop through the URLs
for url, ptime in articles500_info:
    driver.get(url)  # Open the URL
    # Accept cookies if the button exists
    try:
        cookie = driver.find_element(By.XPATH, '//button[contains(@class,"VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 XWZjwc")]')
        cookie.click()
        time.sleep(3)  # Wait for the cookies modal to disappear
    except:
        pass  # Do nothing in the except part
    
    time.sleep(3)  # Allow the page some time to fully load
        
    # Parse the HTML of the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
        
    # Find all the elements that are paragraph HTML elements
    paragraphs = soup.find_all('p')
        
    # Extract the text from each paragraph and append each as a separate data point
    for para in paragraphs:
        para_text = para.get_text()
        articles_500e.append([url, ptime, para_text])

# Close the driver
driver.quit()

# Create a DataFrame and save it to a CSV file
articles_500e_original = pd.DataFrame(articles_500e, columns=['URL', 'Published', 'Text'])

print("Scraping complete. Data saved to articles.csv")
print(articles_500e_original.head())

# quit the browser
driver.quit()

Scraping complete. Data saved to articles.csv
                                                 URL             Published  \
0  https://news.google.com/articles/CBMiSGh0dHBzO...  2024-07-29T10:06:58Z   
1  https://news.google.com/articles/CBMiSGh0dHBzO...  2024-07-29T10:06:58Z   
2  https://news.google.com/articles/CBMiSGh0dHBzO...  2024-07-29T10:06:58Z   
3  https://news.google.com/articles/CBMiSGh0dHBzO...  2024-07-29T10:06:58Z   
4  https://news.google.com/articles/CBMiSGh0dHBzO...  2024-07-29T10:06:58Z   

                                                Text  
0  Price reductions on selected cars, from £250 -...  
1                                         My Profile  
2  A breakdown of the electric Fiat 500's prices ...  
3  The Fiat 500 electric’s price makes it one of ...  
4  With affordable new and used prices, it’s hard...  


### 1.2.2 Cleaning

In [19]:
# copy to clean the data
articles_500e_process = articles_500e_original.copy().drop_duplicates().dropna()
print(len(articles_500e_process))

4417


In [20]:
# clean rows that have only one word in 'Text' column
articles_500e_process = articles_500e_process[articles_500e_process['Text'].str.split().str.len() > 1]

In [21]:
# clean rows that mention date
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'\d{4}-\d{2}-\d{2}', regex=True)]
# mention date in format Month 01, 2023
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{2},\s\d{4}', regex=True)]
# mention date in format 01 Month 2023
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'\d{2}\s[a-zA-Z]+\s\d{4}', regex=True)]
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{1},\s\d{4}', regex=True)]
# mention date in format Month 01, 2023
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{2},\s\d{4}', regex=True)]
# mention date in format 01 Month 2023
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'\d{2}\s[a-zA-Z]+\s\d{4}', regex=True)]
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'[a-zA-Z]+\s\d{1},\s\d{4}', regex=True)]

patterns500e_list  = [
    "chief auto correspondent", "Terms of Service","rejected", "User accounts","ice cream","October report","Bryce Gaton","search keywords", "Bronwyn O'Neill",
    "20-something Joy", "Fusio", "days ago","Deal of the Day","BMW","Air Jordans", "what this bit even is","this is hilarious", "sort of worth writing about",
    "Do The Right Thing","Buggin’ Out","carpenter","Neopolitan", "matching i3", "legend of unspoken value", "WOW","homepage","consent",
    "a good ad","on a dealer lot","Freddie Fazbear’s","Torchinski", "Instagram", "quick reply", "stupid electric door handles","Tyrese Maxey", 
    "NYC is one of the few markets", "rich old folks", "MIL", "housing development", "perfect for her","buying, selling and servicing","power user",
    "888-776-0942", "— Agent009", "FIM","Luca Ciferri","modal window.", "Read: Best Home EV Chargers for 2024", "These technologies are needed", "share this article",
    "These technologies allow","0-60 performance, price", "Ford ",  "Trump", "Read Top Gear’s","Melinda French","Ralph Gilles","Stacy M. Brown",  "felony",
    "Homebuyers","District Attorney", "Oakland", "Toyota", "Police", "Chevrolet", "Send us a letter", "knowledge and connections", "Detroit", "LOS ANGELES", "Democrats",
    "SoCal","Seattle",  "Corsa", "Jeep", "France", "Fiat Topolino","Renault", "Mobilize Duo", "Consider our horses", "Dacia", "full review here", "Volkswagen", "You May Also Like",
    "F-RAM in xEV", "Virtual Twin", "Android Automotive", "The Road to Autonomous", "Twitter", "Ferrari", "Hummer", "Tesla", "Commercial charging cables", "It's a simple but unsettling answer", 
    "Volvo", "New Products & Services", "Acura", "Honda", "Nissan", "See the Best Deals", "your credit situation", "travel photography", "fellow photographers",
    "unlimited access","Enjoy your first month","Digital Camera World", "APR Deals", "Are you driven and customer-focused?","Do you have a natural ability to build relationships?",
    "Stan Miller Inc", "Apartment Manager", "Government Positions", "crime story", "Glenwood Springs", "submit", "google news", "sign up", "summit", "visibility", "sign on", "sign on",
    "try first month", "reporting by", "editing by", "posts by", "Jonathan","thread","Terms & Conditions", "share your info","eDrive Stores","Latest Report Highlights","Zinc Battery",
    "Trade Show","More photo","More latest","View details","Show partners","Share this article","AUBURN HILLS","Breaking News","National Correspondent","@","NNPA","Roosevelt",
    "will make a difference","Legal Ad","Legal Ads","Farm and Ranch","Commercial Print","Work for us","aspen","Printing Press",
    "Manage push notifications","auto connected car news","Site or app","Be the first to post"
]


# combine the patterns
patterns500e = patterns500e_list + patterns600e

# Apply the filters
for pattern in patterns500e:
    articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(pattern, case=False, na=False)]

# clean erows containing 3 dots '...' in a row
articles_500e_process = articles_500e_process[~articles_500e_process['Text'].str.contains(r'\.\.\.', regex=True)]

# clean rows IF (consists of 3 words ) AND (includes  'by')
for index, row in articles_500e_process.iterrows():
    # if there are 3 words in the text and ('by' or 'By') is in the text, drop the row
    if len(row['Text'].split()) == 3 and 'by' in row['Text'].lower():
        articles_500e_process.drop(index, inplace=True)

# drop if only 2 words in the text
articles_500e_process = articles_500e_process[articles_500e_process['Text'].str.split().str.len() > 2]

# clean rows IF (consists of less than 4 words) AND (includes  ':')
for index, row in articles_500e_process.iterrows():
    # if there are less than 4 words in the text and ':' is in the text, drop the row
    if len(row['Text'].split()) <= 6 and ':' in row['Text']:
        articles_500e_process.drop(index, inplace=True)

In [22]:
print(len(articles_500e_original), 'rows in original dataset')
print(len(articles_500e_original) - len(articles_500e_process), 'rows dropped after cleaning')
print(len(articles_500e_process), 'rows left after cleaning')

9366 rows in original dataset
8675 rows dropped after cleaning
691 rows left after cleaning


In [42]:
# Combine paragraphs of the same article
# Group by 'URL' and concatenate text in the same group
articles500_concatenated = articles_500e_process.groupby('URL').agg({'Text': ' '.join, 'Published': 'first'}).reset_index()

# Display the first few rows to verify
print('Number of rows in 500e - Articles: ', len(articles500_concatenated))
print('First 5 rows in 500e - Articles:\n')
print(articles500_concatenated.head())

Number of rows in 500e - Articles:  87
First 5 rows in 500e - Articles:

                                                 URL  \
0  https://news.google.com/articles/CBMiKGh0dHBzO...   
1  https://news.google.com/articles/CBMiLWh0dHBzO...   
2  https://news.google.com/articles/CBMiLmh0dHBzO...   
3  https://news.google.com/articles/CBMiMmh0dHBzO...   
4  https://news.google.com/articles/CBMiNmh0dHBzO...   

                                                Text             Published  
0  Fiat made the 2013 500e specifically for the U...  2023-11-07T08:00:00Z  
1  Save Money. Use the U.S. News Best Price Progr...  2024-07-11T07:00:00Z  
2  2024 Fiat 500e (Photo provided by Fiat) La Dol...  2024-07-24T08:30:00Z  
3  The Fiat 500e not only offers the lowest MSRP ...  2024-05-08T07:00:00Z  
4  See also: FIAT 500e: Electric Icon Arrives in ...  2024-02-15T08:00:00Z  


In [24]:
# Rename the columns for consistency
# rename URL to ID
articles500_concatenated.rename(columns={'URL': 'ID'}, inplace=True)
# Rename Published to Date
articles500_concatenated.rename(columns={'Published': 'Date'}, inplace=True)
# Date to YYYY-MM-DD without time dt.strftime('%Y-%m-%d')
articles500_concatenated['Date'] = pd.to_datetime(articles500_concatenated['Date']).dt.strftime('%Y-%m-%d')

In [25]:
# saving the cleaned data
articles500e = articles500_concatenated.copy()

# 2. Combining Opinion Leaders Data

## 2.1 600e - Opinion Leaders

In [46]:
transcripts600e = pd.read_csv('Transcript_600e.csv')
# rename Transcript to Text
transcripts600e = transcripts600e.rename(columns={'Transcript': 'Text'})
# rename Video ID to ID 
transcripts600e = transcripts600e.rename(columns={'VideoID': 'ID'})

In [48]:
# combine vertically data frames by ID and TEXT
opinionLeaders600e = pd.concat([articles600e, transcripts600e], axis=0)
opinionLeaders600e = opinionLeaders600e.reset_index(drop=True)
print('Number of rows in the 600e - Opinion Leaders: ',len(opinionLeaders600e))
print('First 5 rows of the 600e - Opinion Leaders: \n', opinionLeaders600e.head())

Number of rows in the 600e - Opinion Leaders:  99
First 5 rows of the 600e - Opinion Leaders: 
                                                   ID  \
0  https://news.google.com/articles/CBMiHmh0dHBzO...   
1  https://news.google.com/articles/CBMiJ2h0dHBzO...   
2  https://news.google.com/articles/CBMiJ2h0dHBzO...   
3  https://news.google.com/articles/CBMiKWh0dHBzO...   
4  https://news.google.com/articles/CBMiNGh0dHBzO...   

                                                Text        Date Title  \
0  JAKARTA - Fiat recently revealed their latest ...  2023-12-29   NaN   
1  Customers asked for more crossovers, and Fiat ...  2023-08-21   NaN   
2  \nRelatively affordable to buy. Very useable 2...  2023-09-27   NaN   
3  4.0 out of 5 The ‘Frappuccino' of the small el...  2023-09-22   NaN   
4  There are two specifications available on the ...  2024-03-05   NaN   

  Channel Name  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  


In [29]:
# check if emoty ID and Text exist
print(opinionLeaders600e[opinionLeaders600e['ID'].isnull()])
print(opinionLeaders600e[opinionLeaders600e['Text'].isnull()])
print(opinionLeaders600e[opinionLeaders600e['Date'].isnull()])

Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []
Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []
Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []


In [52]:
# Number of rows in the 600e - Opinion Leaders
print('Number of rows in the 600e - Opinion Leaders: ', len(opinionLeaders600e))

Number of rows in the 600e - Opinion Leaders:  99


In [30]:
# Save to csv
opinionLeaders600e.to_csv('600e - Opinion Leaders.csv', index=False)

## 2.2 500e - Opinion Leaders

In [32]:
transcripts500e = pd.read_csv('Transcript_600e.csv')
# rename Transcript to Text
transcripts500e = transcripts500e.rename(columns={'Transcript': 'Text'})
# rename Video ID to ID 
transcripts500e = transcripts500e.rename(columns={'VideoID': 'ID'})

In [33]:
# combine vertically data frames by ID and TEXT
opinionLeaders500e = pd.concat([articles500e, transcripts500e], axis=0)
opinionLeaders500e = opinionLeaders500e.reset_index(drop=True)
print('Number of rows in the 500e - Opinion Leaders: ',len(opinionLeaders500e))
print('First 5 rows of the 500e - Opinion Leaders: \n', opinionLeaders500e.head())

Number of rows in the 500e - Opinion Leaders:  102
First 5 rows of the 500e - Opinion Leaders: 
                                                   ID  \
0  https://news.google.com/articles/CBMiKGh0dHBzO...   
1  https://news.google.com/articles/CBMiLWh0dHBzO...   
2  https://news.google.com/articles/CBMiLmh0dHBzO...   
3  https://news.google.com/articles/CBMiMmh0dHBzO...   
4  https://news.google.com/articles/CBMiNmh0dHBzO...   

                                                Text        Date Title  \
0  Fiat made the 2013 500e specifically for the U...  2023-11-07   NaN   
1  Save Money. Use the U.S. News Best Price Progr...  2024-07-11   NaN   
2  2024 Fiat 500e (Photo provided by Fiat) La Dol...  2024-07-24   NaN   
3  The Fiat 500e not only offers the lowest MSRP ...  2024-05-08   NaN   
4  See also: FIAT 500e: Electric Icon Arrives in ...  2024-02-15   NaN   

  Channel Name  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  


In [34]:
# check if emoty ID and Text exist
print(opinionLeaders500e[opinionLeaders500e['ID'].isnull()])
print(opinionLeaders500e[opinionLeaders500e['Text'].isnull()])
print(opinionLeaders500e[opinionLeaders500e['Date'].isnull()])

Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []
Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []
Empty DataFrame
Columns: [ID, Text, Date, Title, Channel Name]
Index: []


In [51]:
# Number of rows in the 500e - Opinion Leaders
print('Number of rows in the 500e - Opinion Leaders: ', len(opinionLeaders500e))

Number of rows in the 500e - Opinion Leaders:  102


In [ ]:
# Save to csv
opinionLeaders500e.to_csv('500e - Opinion Leaders.csv', index=False)